In [1]:
import numpy as np
import pandas as pd

In [2]:
data_path = "../../dataset/3-5modified.csv"
res_path = "result-tfidf1.csv"

In [3]:
df = pd.read_csv(data_path)

In [4]:
df.head()

,Date,Time,User1 ID,User2 ID,text,status,text_processed
0,2021-03-04,10:17,1,2,bye bye dear bajaj i got some better work to ...,1,bye bye dear bajaj get good work watch ur funn...
1,2022-01-24,04:27,1,2,Haha your so funny you sit on wikipedia all da...,1,haha funny sit wikipedia day make sure everyth...
2,2021-02-03,02:19,1,2,My problem is people talking out of their asse...,1,problem people talk ass fault people stupid ma...
3,2021-09-04,08:54,1,2,Article updated,0,article update
4,2021-07-21,13:15,1,2,Well arent you phucking special Its easy to ha...,1,well arent phucking special easy thing still l...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90356 entries, 0 to 90355
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            90356 non-null  object
 1   Time            90356 non-null  object
 2   User1 ID        90356 non-null  int64 
 3   User2 ID        90356 non-null  int64 
 4   text            90356 non-null  object
 5   status          90356 non-null  int64 
 6   text_processed  90307 non-null  object
dtypes: int64(3), object(4)
memory usage: 4.8+ MB


In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

In [7]:
df['status'].unique()

array([1, 0])

In [8]:
# df['status_num'] = df['status'].map({'not_cyberbullying':0, 'gender':1, 'religion':2, 'other_cyberbullying':3,
#        'age':4, 'ethnicity':5})
df['status_num'] = df['status']

In [9]:
X = df['text_processed']
y = df['status_num']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [11]:
y_train_exp = np.expand_dims(np.array(y_train),axis=-1)
y_test_exp = np.expand_dims(np.array(y_test),axis=-1)

In [12]:
cv = TfidfVectorizer()

cv.fit(X_train.values.astype('U')) # str will also work
X_train_trans = cv.transform(X_train.values.astype('U'))
X_test_trans = cv.transform(X_test.values.astype('str'))

In [41]:
# !where python

/Applications/anaconda3/envs/miscEnv/bin/python


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve

In [15]:
# lr = LogisticRegression()
svc = SVC(kernel='sigmoid',gamma=1.0)
dtc = DecisionTreeClassifier(max_depth=5)
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()
knc = KNeighborsClassifier()
rf = RandomForestClassifier()
abc = AdaBoostClassifier(n_estimators=50, random_state = 32)
bc = BaggingClassifier(n_estimators=50, random_state=32)
etc = ExtraTreesClassifier(n_estimators = 50, random_state = 32)
gbdt = GradientBoostingClassifier(n_estimators = 50,random_state = 32)
xgb = XGBClassifier(n_estimators=50, random_state = 32)

In [20]:
mdls = {
'svc':svc, # too much time
'dtc':dtc, 
'mnb':mnb,
# 'gnb':gnb, # sparse data issue
'bnb':bnb,
'knc':knc,
'rf':rf,
'abc':abc,
'bc':bc, # too much time
'etc':etc, # too much time
'gbdt':gbdt,
'xgb':xgb,
}

In [21]:
# # mdls = {
# #     # "lr":lr,
# #     "mnb":mnb,
# #     "knc":knc,
# #     "dtc":dtc,
# #     "abc":abc,
# #     "gbdt":gbdt,
# #     "xgb":xgb,
# #     "bc":bc,
# #     "svc":svc,
# #     "etc":etc,
# # }

# mdls = {
# # 'svc':svc, # too much time
# 'dtc':dtc, 
# # 'mnb':mnb,
# # 'gnb':gnb, # won't work
# 'bnb':bnb,
# # 'knc':knc,
# 'rf':rf,
# # 'abc':abc,
# # 'bc':bc, # too much time
# # 'etc':etc, # too much time
# # 'gbdt':gbdt,
# # 'xgb':xgb,
# }


In [17]:
xgb.fit(X_train_trans,y_train)
y_pred2 = xgb.predict(X_test_trans)

In [18]:
print(accuracy_score(y_test,y_pred2))


0.8244798583444002


In [24]:
def predict_mdl(mdl, X_train, y_train, X_test, y_test):
    """Trains model and returns (accuracy,precision)"""
    mdl.fit(X_train, y_train)
    y_preds = mdl.predict(X_test)

    # print('hi')
    print(accuracy_score(y_test,y_preds))
    # print(confusion_matrix(y_test,y_preds))

    return accuracy_score(y_preds, y_test)

In [25]:
names = []
accuracies = []
# precisions = []
for name, mdl in mdls.items():
    print()
    print("---===---"*5)
    print(name)
    acc = predict_mdl(mdl, X_train_trans, y_train, X_test_trans, y_test)
    print(acc)
    names.append(name)
    accuracies.append(acc)



---===------===------===------===------===---
svc
0.867751217352811
0.867751217352811

---===------===------===------===------===---
dtc
0.729858344400177
0.729858344400177

---===------===------===------===------===---
mnb
0.8174524125719345
0.8174524125719345

---===------===------===------===------===---
bnb
0.6968791500664011
0.6968791500664011

---===------===------===------===------===---
knc
0.8063302346170872
0.8063302346170872

---===------===------===------===------===---
rf
0.8487162461266047
0.8487162461266047

---===------===------===------===------===---
abc


/Users/suraj/research/cyber-bullying/.venv/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.7996901283753873
0.7996901283753873

---===------===------===------===------===---
bc
0.8516489597166887
0.8516489597166887

---===------===------===------===------===---
etc
0.8543050022133688
0.8543050022133688

---===------===------===------===------===---
gbdt
0.7679836210712705
0.7679836210712705


In [56]:
times

[43.594560861587524,
 0.0036039352416992188,
 0.014068841934204102,
 0.012914896011352539,
 22.143911123275757,
 1.5198159217834473,
 0.07214784622192383,
 0.9737379550933838,
 1.8291330337524414,
 0.018728017807006836]

In [57]:
res_df1 = pd.DataFrame({"models_tfidf-1":names,"accuracy_tfidf-1":accuracies, 'time-tfidf-1':times})
res_df1

,models_tfidf-1,accuracy_tfidf-1,time-tfidf-1
0,svc,0.867751,43.594561
1,dtc,0.729858,0.003604
2,mnb,0.817452,0.014069
3,bnb,0.696879,0.012915
4,knc,0.806330,22.143911
5,rf,0.848716,1.519816
6,abc,0.799690,0.072148
7,bc,0.851649,0.973738
8,etc,0.854305,1.829133
9,gbdt,0.767984,0.018728


In [59]:
res_df1.to_csv(res_path, index=False)

In [19]:
# note time taken by model to predict 
import time

start = time.time()
xgb.predict(X_test_trans)
end = time.time()
print(end-start)

0.12280821800231934


In [ ]:
# note time taken by model to predict 
import time

start = time.time()
xgb.predict(X_test_trans)
end = time.time()
print(end-start)

0.12280821800231934


In [51]:
# note time taken by model to predict 
import time

def note_prediction_time(mdl, X_test):
    start = time.time()
    mdl.predict(X_test)
    end = time.time()
    return end-start

In [54]:
# # note_prediction_time(rf, X_test_trans)
# times = []
# for name, mdl in mdls.items():
#     print('---===---'*5)
#     print(name)
#     tim = note_prediction_time(mdl, X_test_trans)
#     print(tim)
#     times.append(tim)
#     print('\n')

In [28]:
# names1 = []
# accuracies1 = []
# for name, mdl in mdls.items():
#     print()
#     print("---===---"*5)
#     print(name)
#     acc = predict_mdl(mdl, X_train_trans, y_train, X_test_trans, y_test)
#     print(acc)
#     names1.append(name)
#     accuracies1.append(acc)


In [16]:
# times

In [29]:
# res_df = pd.concat([res_df,res_df2],ignore_index=True)


In [30]:
# res_df.to_csv(res_path)
# res_df

In [31]:
# save tifidf vectorizer
import pickle
pickle.dump(cv, open('tfidf_vectorizer.pkl', 'wb'))

In [20]:
import pickle
# save model above 80% accuracy using pickle
pickle.dump(svc, open('xgb-tfidf.pkl', 'wb'))

In [44]:
# load tifdf vectorizer
tf = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))

# load model
model = pickle.load(open('svc-tfidf.pkl', 'rb'))

In [45]:
y_pred2 = model.predict(X_test_trans)

In [46]:
print(accuracy_score(y_test,y_pred2))

0.867751217352811


In [60]:
mdl_path = 'models/'

for name, mdl in mdls.items():
    print('---===---'*5)
    print(name)
    p = f'{mdl_path}{name}-tfidf.pkl'
    pickle.dump(mdl, open(p, 'wb'))
    
    

---===------===------===------===------===---
svc
---===------===------===------===------===---
dtc
---===------===------===------===------===---
mnb
---===------===------===------===------===---
bnb
---===------===------===------===------===---
knc
---===------===------===------===------===---
rf
---===------===------===------===------===---
abc
---===------===------===------===------===---
bc
---===------===------===------===------===---
etc
---===------===------===------===------===---
gbdt


In [62]:
pd.read_csv(res_path)

,models_tfidf-1,accuracy_tfidf-1,predict_time-tfidf-1
0,svc,0.867751,43.594561
1,dtc,0.729858,0.003604
2,mnb,0.817452,0.014069
3,bnb,0.696879,0.012915
4,knc,0.806330,22.143911
5,rf,0.848716,1.519816
6,abc,0.799690,0.072148
7,bc,0.851649,0.973738
8,etc,0.854305,1.829133
9,gbdt,0.767984,0.018728


In [63]:
len(X_test)

18072

In [77]:
X_test_trans[0]

<1x123712 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [78]:
start = time.time()
mnb.predict(X_test_trans[0].reshape(1,-1))
time.time()-start

0.00741124153137207

In [23]:
res = pd.read_csv(res_path)
res

,models_tfidf-1,accuracy_tfidf-1,predict_time-tfidf-1
0,svc,0.867751,43.594561
1,dtc,0.729858,0.003604
2,mnb,0.817452,0.014069
3,bnb,0.696879,0.012915
4,knc,0.806330,22.143911
5,rf,0.848716,1.519816
6,abc,0.799690,0.072148
7,bc,0.851649,0.973738
8,etc,0.854305,1.829133
9,gbdt,0.767984,0.018728
